In [16]:
%load_ext kedro.extras.extensions.ipython
%reload_kedro /Users/yeungadrian/Documents/repo/TimeSeries

The kedro.extras.extensions.ipython extension is already loaded. To reload it, use:
  %reload_ext kedro.extras.extensions.ipython
2022-06-25 19:19:39,034 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2022-06-25 19:19:39,118 - root - INFO - ** Kedro project timeseries
2022-06-25 19:19:39,120 - root - INFO - Defined global variable `context`, `session`, `catalog` and `pipelines`


## To Do:
- Add ability to leverage
  - Cost of leverage
  - Margin
- Benchmark 

## Libraries

In [46]:
import numpy as np
import pandas as pd
from pydantic import BaseModel
from typing import List, TypeVar, Optional, Dict
from importlib import reload

import portfolio
reload(portfolio)

from portfolio import DataLoader, Portfolio


## Data

In [47]:
prices = catalog.load("fund_prices")
prices

2022-06-25 19:45:06,671 - kedro.io.data_catalog - INFO - Loading data from `fund_prices` (ParquetDataSet)...


,date,ABMD,ATVI,AMD,AMZN,AAPL,DTE,EBAY
0,2020-12-31,324.200,92.850000,91.710,3256.93,132.690000,121.410000,50.250000
1,2020-12-30,323.920,91.580000,92.290,3285.85,133.720000,119.660000,50.550000
2,2020-12-29,320.930,91.370000,90.620,3322.00,134.870000,119.240000,50.860000
3,2020-12-28,312.910,91.430000,91.600,3283.96,136.690000,119.230000,50.240000
4,2020-12-24,303.410,90.960000,91.810,3172.69,131.970000,119.720000,50.120000
...,...,...,...,...,...,...,...,...
5322,1999-11-05,11.315,1.148739,10.625,64.94,0.680100,13.626682,6.904808
5323,1999-11-04,10.470,1.091975,10.315,63.06,0.643981,13.360490,6.733871
5324,1999-11-03,10.250,1.083012,10.655,65.81,0.627654,13.626682,6.583654
5325,1999-11-02,10.315,1.064339,10.280,66.44,0.618028,13.626682,6.801210


## Backtesting Portfolio

In [48]:
prices = DataLoader().prepare_data(prices)

In [49]:
prices

,date,ABMD,ATVI,AMD,AMZN,AAPL,DTE,EBAY
0,1999-11-01,10.250,1.050148,10.155,69.13,0.597773,13.235224,7.096464
1,1999-11-02,10.315,1.064339,10.280,66.44,0.618028,13.626682,6.801210
2,1999-11-03,10.250,1.083012,10.655,65.81,0.627654,13.626682,6.583654
3,1999-11-04,10.470,1.091975,10.315,63.06,0.643981,13.360490,6.733871
4,1999-11-05,11.315,1.148739,10.625,64.94,0.680100,13.626682,6.904808
...,...,...,...,...,...,...,...,...
5322,2020-12-24,303.410,90.960000,91.810,3172.69,131.970000,119.720000,50.120000
5323,2020-12-28,312.910,91.430000,91.600,3283.96,136.690000,119.230000,50.240000
5324,2020-12-29,320.930,91.370000,90.620,3322.00,134.870000,119.240000,50.860000
5325,2020-12-30,323.920,91.580000,92.290,3285.85,133.720000,119.660000,50.550000


In [50]:
external_data = {
    "codes": ["EBAY", "AAPL"],
    "amounts": [30,30],
    "start_date": "2012-12-27",
    "end_date": "2019-12-31",
    "timeseries": prices,
    "rebalance": True,
    "rebalance_frequency": "m"
}

In [51]:
prices

,date,ABMD,ATVI,AMD,AMZN,AAPL,DTE,EBAY
0,1999-11-01,10.250,1.050148,10.155,69.13,0.597773,13.235224,7.096464
1,1999-11-02,10.315,1.064339,10.280,66.44,0.618028,13.626682,6.801210
2,1999-11-03,10.250,1.083012,10.655,65.81,0.627654,13.626682,6.583654
3,1999-11-04,10.470,1.091975,10.315,63.06,0.643981,13.360490,6.733871
4,1999-11-05,11.315,1.148739,10.625,64.94,0.680100,13.626682,6.904808
...,...,...,...,...,...,...,...,...
5322,2020-12-24,303.410,90.960000,91.810,3172.69,131.970000,119.720000,50.120000
5323,2020-12-28,312.910,91.430000,91.600,3283.96,136.690000,119.230000,50.240000
5324,2020-12-29,320.930,91.370000,90.620,3322.00,134.870000,119.240000,50.860000
5325,2020-12-30,323.920,91.580000,92.290,3285.85,133.720000,119.660000,50.550000


In [52]:
mock_portfolio = Portfolio(**external_data)

In [53]:
mock_portfolio.backtest_strategy()

,EBAY,AAPL,portfolio,date
0,30.000000,30.000000,60.000000,2012-12-27
1,29.713661,29.681350,59.395011,2012-12-28
2,30.422171,30.996752,61.418923,2012-12-31
1,32.270476,31.682214,63.952690,2013-01-02
2,31.583878,31.282076,62.865954,2013-01-03
...,...,...,...,...
17,88.028200,91.850086,179.878286,2019-12-24
18,88.125442,93.672419,181.797861,2019-12-26
19,87.687853,93.636877,181.324730,2019-12-27
20,87.031471,94.192624,181.224095,2019-12-30


In [54]:
something = mock_portfolio.backtest_strategy()


In [55]:
something['date'] = pd.to_datetime(something['date'])

In [56]:
something

,EBAY,AAPL,portfolio,date
0,30.000000,30.000000,60.000000,2012-12-27
1,29.713661,29.681350,59.395011,2012-12-28
2,30.422171,30.996752,61.418923,2012-12-31
1,32.270476,31.682214,63.952690,2013-01-02
2,31.583878,31.282076,62.865954,2013-01-03
...,...,...,...,...
17,88.028200,91.850086,179.878286,2019-12-24
18,88.125442,93.672419,181.797861,2019-12-26
19,87.687853,93.636877,181.324730,2019-12-27
20,87.031471,94.192624,181.224095,2019-12-30


In [57]:
idx = pd.date_range(external_data['start_date'],external_data['end_date'],freq='D')

In [58]:
pd.DatetimeIndex(something.date)

DatetimeIndex(['2012-12-27', '2012-12-28', '2012-12-31', '2013-01-02',
               '2013-01-03', '2013-01-04', '2013-01-07', '2013-01-08',
               '2013-01-09', '2013-01-10',
               ...
               '2019-12-17', '2019-12-18', '2019-12-19', '2019-12-20',
               '2019-12-23', '2019-12-24', '2019-12-26', '2019-12-27',
               '2019-12-30', '2019-12-31'],
              dtype='datetime64[ns]', name='date', length=1765, freq=None)

In [64]:
something = something.set_index(pd.DatetimeIndex(something.date))

In [65]:
something.reindex(idx, fill_value=0)

,EBAY,AAPL,portfolio,date
2012-12-27,30.000000,30.000000,60.000000,2012-12-27 00:00:00
2012-12-28,29.713661,29.681350,59.395011,2012-12-28 00:00:00
2012-12-29,0.000000,0.000000,0.000000,0
2012-12-30,0.000000,0.000000,0.000000,0
2012-12-31,30.422171,30.996752,61.418923,2012-12-31 00:00:00
...,...,...,...,...
2019-12-27,87.687853,93.636877,181.324730,2019-12-27 00:00:00
2019-12-28,0.000000,0.000000,0.000000,0
2019-12-29,0.000000,0.000000,0.000000,0
2019-12-30,87.031471,94.192624,181.224095,2019-12-30 00:00:00
